# Introduction
The following code will analyze the `name` column for feature engineering  
**TODOS**:  
* [ ] Hard-coded fuzzy matching
* [ ] fuzzywuzzy library
* [ ] comparison of performance
* [ ] implement into preprocessing code